## **Import libraries and load data from database**

In [1]:
!pip install nltk
!pip install plotly

In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import pickle
import nltk
import sys

# import relevant functions/modules from the nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# import relevant functions/modules from the sklearn

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [4]:
df = pd.read_csv('df.csv')
del df['Unnamed: 0']

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
database_filepath = "/content/disaster_response_db.db"
engine = create_engine('sqlite:///' + database_filepath)
table_name = "disaster_response_db_table"
df = pd.read_sql_table(table_name,engine)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X = df['message']
Y = df.iloc[:,4:]

#### Tokenize text

In [7]:
def tokenize(text, url_place_holder_string = "urlplaceholder"):
    """Normalize, replace URLs, tokenize and lemmatize text string
    
    Args:
    text: string. String containing message for processing
       
    Returns:
    clean_tokens: list of strings. List containing processed input
    """
    # Convert text to lowercase and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    #
    url_regex = 'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)'
    
    detected_urls = re.findall(url_regex, text)
    
    # Replace url with a url placeholder string
    for detected_url in detected_urls:
        text = text.replace(detected_url, url_place_holder_string)

    # Tokenize words
    tokens = word_tokenize(text)
    
    # Stem word tokens and remove stop words
    lemmatizer = nltk.WordNetLemmatizer()

    clean_tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]
    return clean_tokens

#### Build a machine learning pipeline

In [8]:
initial_pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier())),
    ])

#### Train the model

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42)
model = initial_pipeline.fit(X_train, Y_train)

#### Test your model

In [10]:
Y_prediction_test = model.predict(X_test)
Y_pred = pd.DataFrame(Y_prediction_test, columns = Y_test.columns)
for column in Y_test.columns:
        print('Model Performance with Category: {}'.format(column))
        print(classification_report(Y_test[column],Y_pred[column]))

Model Performance with Category: related
              precision    recall  f1-score   support

           0       0.75      0.28      0.40      1569
           1       0.81      0.97      0.88      4938

    accuracy                           0.80      6507
   macro avg       0.78      0.62      0.64      6507
weighted avg       0.80      0.80      0.77      6507

Model Performance with Category: request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5403
           1       0.87      0.44      0.59      1104

    accuracy                           0.89      6507
   macro avg       0.88      0.71      0.76      6507
weighted avg       0.89      0.89      0.88      6507

Model Performance with Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6481
           1       0.00      0.00      0.00        26

    accuracy                           1.00      6507
   macr

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6467
           1       0.00      0.00      0.00        40

    accuracy                           0.99      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      0.99      0.99      6507

Model Performance with Category: hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6436
           1       0.00      0.00      0.00        71

    accuracy                           0.99      6507
   macro avg       0.49      0.50      0.50      6507
weighted avg       0.98      0.99      0.98      6507

Model Performance with Category: shops
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6472
           1       0.00      0.00      0.00        35

    accuracy                           0.99      6507
   macro avg       0.50      0.50      0.50   

#### Improve model

##### Use GridSearchCV for optimization of parameters

In [11]:
parameters = {'vect__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators': [10, 20, 40], 
              'clf__estimator__min_samples_split':[2, 5, 10]}

cv = GridSearchCV(initial_pipeline, param_grid = parameters, scoring = 'f1_micro', verbose = 10, n_jobs=-1)

# Find best parameters
np.random.seed(81)
tuned_model = cv.fit(X_train, Y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


In [12]:
tuned_model.best_estimator_

Pipeline(steps=[('vect',
                 CountVectorizer(min_df=5,
                                 tokenizer=<function tokenize at 0x7fc7579d9950>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(min_samples_split=10,
                                                                        n_estimators=40)))])

In [13]:
tuned_model.best_params_

{'clf__estimator__min_samples_split': 10,
 'clf__estimator__n_estimators': 40,
 'tfidf__use_idf': True,
 'vect__min_df': 5}

#### Test new improved model

In [14]:
Y_prediction_test = tuned_model.predict(X_test)

In [15]:
Y_pred = pd.DataFrame(Y_prediction_test, columns = Y_test.columns)
for column in Y_test.columns:
        print('Model Performance with Category: {}'.format(column))
        print(classification_report(Y_test[column],Y_pred[column]))

Model Performance with Category: related
              precision    recall  f1-score   support

           0       0.76      0.28      0.41      1569
           1       0.81      0.97      0.88      4938

    accuracy                           0.81      6507
   macro avg       0.79      0.63      0.65      6507
weighted avg       0.80      0.81      0.77      6507

Model Performance with Category: request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5403
           1       0.86      0.48      0.62      1104

    accuracy                           0.90      6507
   macro avg       0.88      0.73      0.78      6507
weighted avg       0.90      0.90      0.89      6507

Model Performance with Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6481
           1       0.00      0.00      0.00        26

    accuracy                           1.00      6507
   macr

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

              precision    recall  f1-score   support

           0       0.90      0.94      0.92      4693
           1       0.83      0.72      0.77      1814

    accuracy                           0.88      6507
   macro avg       0.87      0.83      0.85      6507
weighted avg       0.88      0.88      0.88      6507

Model Performance with Category: floods
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      5973
           1       0.91      0.54      0.68       534

    accuracy                           0.96      6507
   macro avg       0.94      0.77      0.83      6507
weighted avg       0.96      0.96      0.95      6507

Model Performance with Category: storm
              precision    recall  f1-score   support

           0       0.96      0.98      0.97      5883
           1       0.76      0.63      0.69       624

    accuracy                           0.95      6507
   macro avg       0.86      0.81      0.83      

#### We see good improvement in the F1 score after tuning, which is what we were optimizing for.

#### Trying different ML algorithm: ADABoost Classifier

In [16]:
new_pipeline= Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

parameters2 = {'clf__estimator__learning_rate': [0.01, 0.02, 0.05],
               'clf__estimator__n_estimators': [10, 20, 40]}

cv2 = GridSearchCV(new_pipeline, param_grid = parameters2, scoring = 'f1_micro', verbose = 10, n_jobs=-1)

# Find best parameters
np.random.seed(81)
tuned_model2 = cv2.fit(X_train, Y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


In [17]:
tuned_model2.best_params_

{'clf__estimator__learning_rate': 0.01, 'clf__estimator__n_estimators': 10}

#### Test different ML model

In [18]:
Y_prediction_test = tuned_model2.predict(X_test)

In [19]:
Y_pred = pd.DataFrame(Y_prediction_test, columns = Y_test.columns)
for column in Y_test.columns:
        print('Model Performance with Category: {}'.format(column))
        print(classification_report(Y_test[column],Y_pred[column]))

Model Performance with Category: related
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1569
           1       0.76      1.00      0.86      4938

    accuracy                           0.76      6507
   macro avg       0.38      0.50      0.43      6507
weighted avg       0.58      0.76      0.65      6507

Model Performance with Category: request
              precision    recall  f1-score   support

           0       0.89      0.95      0.92      5403
           1       0.62      0.40      0.49      1104

    accuracy                           0.86      6507
   macro avg       0.75      0.68      0.70      6507
weighted avg       0.84      0.86      0.84      6507

Model Performance with Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6481
           1       0.00      0.00      0.00        26

    accuracy                           1.00      6507
   macr

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6226
           1       0.00      0.00      0.00       281

    accuracy                           0.96      6507
   macro avg       0.48      0.50      0.49      6507
weighted avg       0.92      0.96      0.94      6507

Model Performance with Category: weather_related
              precision    recall  f1-score   support

           0       0.77      0.99      0.87      4693
           1       0.91      0.22      0.36      1814

    accuracy                           0.78      6507
   macro avg       0.84      0.61      0.61      6507
weighted avg       0.81      0.78      0.72      6507

Model Performance with Category: floods
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      5973
           1       0.91      0.39      0.55       534

    accuracy                           0.95      6507
   macro avg       0.93      0.69      

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### The random forest model has better performance

In [20]:
model_filename = 'classifier.pkl'
with open(model_filename, 'wb') as file:  
    pickle.dump(tuned_model, file)